In [1]:
import warnings
warnings.filterwarnings('ignore')
import astropy
from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import WMAP9 as cosmo
import pandas as pd
import numpy as np
import gzip
import math
import matplotlib.pyplot as plt
import linmix

In [2]:
pabdatazcut = fits.open('pabdatazcut.fits')
sortedpab = sorted(pabdatazcut[1].data , key = lambda data: data['PAB_FLUX'] , reverse = True )

data_dict= dict()
for obj in sortedpab:
    for key in ['FIELD', 'ID', 'RA', 'DEC' , 'Z_50', 'Z_84','Z_16' , 'PAB_FLUX', 'PAB_FLUX_ERR']:
        data_dict.setdefault(key, list()).append(obj[key])

sortedpabdf = pd.DataFrame(data_dict)
sortedpabdf['Z_ERR']= ((sortedpabdf['Z_84'] - sortedpabdf['Z_50']) + (sortedpabdf['Z_50'] - sortedpabdf['Z_16'])) / (2 * sortedpabdf['Z_50'])
sortedpabdf['OBS_PAB'] = 12820 * (1 + sortedpabdf['Z_50'])
sortedpabdf.loc[sortedpabdf['FIELD'] == "ERS" , 'FIELD'] = "ERSPRIME"
sortedpabdf = sortedpabdf[['FIELD' , 'ID' , 'RA' , 'DEC' , 'Z_50' , 'Z_ERR' , 'PAB_FLUX' , 'PAB_FLUX_ERR' , 'OBS_PAB']]

In [3]:
sortedpabdf.to_csv('sortedpabdf.csv',index=False)

In [4]:
good = [6 , 9 , 12 , 14 , 15 , 18 , 21 , 24 , 29 , 30 , 35 , 36 , 37 , 46 , 48 , 51 , 52 , 55 , 58 , 70 , 71 , 72 , 74 , 75 , 78 , 80 , 81 , 84 , 89 , 97 , 98 , 102 ,  112 , 118 , 119 , 120 , 125 , 126 , 128 , 129 , 132 , 138 , 147 , 148 , 150 , 158 , 159 , 160 , 163 , 166 , 181 , 184 , 186 , 189 , 193 , 194 , 196 , 206 , 213 , 215 , 235 , 244 , 257 , 259 , 265 , 275 , 280 , 284 , 285 , 290 , 306 , 319 , 326 , 339 , 347 , 365 ]
gooddf50 = sortedpabdf.loc[good]

GNdf = gooddf50[gooddf50['FIELD'] == 'GN1']
GNdf = GNdf.append(gooddf50[gooddf50['FIELD'] == 'GN2'])
GNdf = GNdf.append(gooddf50[gooddf50['FIELD'] == 'GN3'])
GNdf = GNdf.append(gooddf50[gooddf50['FIELD'] == 'GN4'])
GNdf = GNdf.append(gooddf50[gooddf50['FIELD'] == 'GN5'])
GNdf = GNdf.append(gooddf50[gooddf50['FIELD'] == 'GN7'])
GNbad = [38114 , 21829 , 22158 , 25434 , 35356 , 19075 , 22583 , 22933 , 26542 , 33015 , 37721 , 32992 , 36624]
GNdf = GNdf[GNdf.ID.isin(GNbad) == False]

GSdf = gooddf50[gooddf50['FIELD'] == 'GS1']
GSdf = GSdf.append(gooddf50[gooddf50['FIELD'] == 'GS2'])
GSdf = GSdf.append(gooddf50[gooddf50['FIELD'] == 'GS3'])
GSdf = GSdf.append(gooddf50[gooddf50['FIELD'] == 'GS4'])
GSdf = GSdf.append(gooddf50[gooddf50['FIELD'] == 'GS5'])
GSdf = GSdf.append(gooddf50[gooddf50['FIELD'] == 'ERSPRIME'])
GSbad = [39505 , 41397 , 43823 , 43401 , 36971 , 26915 , 37739 , 36001 , 42866 , 39042 , 44831 , 44959 , 34363 , 36624 , 20623 , 24491 , 43700]
GSdf = GSdf[GSdf.ID.isin(GSbad) == False]

fulldf = pd.concat([GNdf , GSdf])
fulldf['DL'] = cosmo.luminosity_distance(fulldf['Z_50'])
fulldf = fulldf[fulldf.ID != 47421]
sample = np.unique([19221 , 15610 , 34456 , 34157 , 33397 , 33511 , 34368 , 34077 , 33249 , 45518 , 49771 , 37720 , 41882 , 27438 ,
 27549 , 26639 , 26696 , 25632 , 43071 , 39634 , 44465 , 37683 , 19221 , 15610 , 18157 , 34456 , 34157 , 33511 , 34368 , 35455 , 24611 , 33249 ,
19221 , 15610 , 18157 , 34456 , 33511 , 34368 , 35455 , 24611 , 37720 , 41882 , 27438 , 26639 , 26696 , 26646 , 21693 , 42536 , 65433 , 39564 , 27535 , 26559])
print(len(sample))
fulldf = fulldf[fulldf.ID.isin(sample) == True]
#display(fulldf)
#print(len(fulldf))
fulldf.to_csv('fulldf.csv',index=False)

32


In [5]:
fulldf = pd.read_csv('fulldf.csv')

In [5]:
GNtable = Table.read('GOODSN', path = 'data') 
GStable = Table.read('GOODSS', path = 'data')

data_dictn= dict()
for obj in GNtable:
    for key in ['id3DHST' , 'RA', 'DEC', 'ir_SFR-ladder_total' , 'ir_SFR-ladder_total_Av' ,'gf_n_j', 'gf_dn_j' , 'gf_f_j' , 'gf_re_j' , 'gf_dre_j' , 'td_Av', 'ACS_F435W_FLUX' , 'ACS_F775W_FLUX' , 'restalt_restUjohnson' , 'restalt_restVjohnson'  , 'td_flag1' , 'td_z_best' , 'td_z_spec' , 'td_z_best_s' , 'td_z_peak_phot' ,'td_lmass', 'td_sfr' , 'td_sfr_IR' , 'td_sfr_UV', 'ir_z_ir' ,'ir_SFR-UV_corr' , 'ir_SFR-UV_corr_Error','ir_SFR-IR', 'ir_UV_beta' , 'ir_A_UV_280']:
        data_dictn.setdefault(key, list()).append(obj[key])

GNtabledf = pd.DataFrame(data_dictn)
GNmiddf = GNtabledf[(GNtabledf['td_z_best'] <= 1)&(GNtabledf['td_z_best'] >= .5)]
GNtabledf = GNtabledf[GNtabledf['td_z_best'] <= .31]
GNtabledf = GNtabledf[GNtabledf['td_z_best'] >= .1]
data_dicts= dict()
for obj in GStable:
    for key in ['id3DHST' , 'RA', 'DEC', 'ir_SFR-ladder_total' , 'ir_SFR-ladder_total_Av', 'gf_n_j', 'gf_dn_j' , 'gf_f_j' , 'gf_re_j' , 'gf_dre_j' , 'td_Av' , 'ACS_F435W_FLUX' , 'ACS_F775W_FLUX' , 'restUjohnson' , 'restVjohnson' , 'td_flag1' , 'td_z_best' , 'td_z_spec', 'td_z_best_s', 'td_z_peak_phot', 'td_lmass' , 'td_sfr' , 'td_sfr_IR' , 'td_sfr_UV' , 'ir_z_ir' , 'ir_SFR-UV_corr', 'ir_SFR-UV_corr_Error', 'ir_SFR-IR', 'ir_UV_beta', 'ir_A_UV_280']:
        data_dicts.setdefault(key, list()).append(obj[key])

GStabledf = pd.DataFrame(data_dicts)
GSmiddf = GStabledf[(GStabledf['td_z_best'] <= 1)&(GStabledf['td_z_best'] >= .5)]
GStabledf = GStabledf[GStabledf['td_z_best'] <= .31]



In [4]:
display(GNtable)
#display(GNtabledf)

<Table length=35445>
  ID             IAU_Name                 RA      ... dim_vj_D dim_err_vj_D
int64            bytes30               float64    ... float64    float32   
----- ------------------------------ ------------ ... -------- ------------
    1 CANDELS_GDN_J123636.5+620530.4 189.15216178 ... 0.639391       0.1653
    2 CANDELS_GDN_J123635.7+620531.3 189.14883356 ... 0.639391       0.1653
    3 CANDELS_GDN_J123636.3+620531.7 189.15133834 ... 0.639391       0.1653
    4 CANDELS_GDN_J123635.9+620531.6 189.14960603 ... 0.639391       0.1653
    5 CANDELS_GDN_J123635.5+620533.0 189.14808454 ... 0.639391       0.1653
    6 CANDELS_GDN_J123635.7+620533.7 189.14870539 ... 0.639391       0.1653
    7 CANDELS_GDN_J123635.1+620534.4 189.14636359 ... 0.639391       0.1653
    8 CANDELS_GDN_J123634.8+620536.2 189.14509981 ... 0.639391       0.1653
    9 CANDELS_GDN_J123634.6+620536.7 189.14428042 ... 0.639391       0.1653
   10 CANDELS_GDN_J123637.2+620538.8 189.15491217 ... 0.639391       0.1653
  ...                            ...          ... ...      ...          ...
35442 CANDELS_GDN_J123646.0+622025.8 189.19168439 ... 0.639391       0.1653
35443 CANDELS_GDN_J123724.6+622051.6 189.35238358 ... 0.639391       0.1653
35444 CANDELS_GDN_J123719.5+622100.5 189.33131392 ... 0.639391       0.1653
35445 CANDELS_GDN_J123646.2+622022.5 189.19263691 ... 0.639391       0.1653
35446 CANDELS_GDN_J123657.4+622111.8 189.23914674 ... 0.639391       0.1653
35447 CANDELS_GDN_J123719.4+622023.8 189.33066692 ... 0.639391       0.1653
35448 CANDELS_GDN_J123658.4+622055.9 189.24320511 ... 0.639391       0.1653
35449 CANDELS_GDN_J123728.5+622034.4 189.36866314 ... 0.639391       0.1653
35450 CANDELS_GDN_J123643.2+621914.6 189.17987934 ... 0.639391       0.1653
35451 CANDELS_GDN_J123714.1+622012.7  189.3086024 ... 0.639391       0.1653

In [7]:
barromiddf = pd.concat([GNmiddf , GSmiddf])
barromiddf.to_csv('barromiddf.csv', index = False)

In [7]:
idmergedf = pd.merge(GNdf, GNtabledf , how = 'inner' , left_on = 'ID' , right_on = 'id3DHST')
idmergedf.drop('RA_y', axis=1, inplace=True)
idmergedf.drop('DEC_y', axis=1, inplace=True)
idmergedf.loc[5 , 'Z_50'] = 0.21129
idmergedf.loc[5 , 'PAB_FLUX'] = 27.5076294
idmergedf.loc[5 , 'PAB_FLUX_ERR'] = 2.86222911
idmergedf['DL'] = cosmo.luminosity_distance(idmergedf['Z_50'])
idmergedf['PAB_L'] = idmergedf['PAB_FLUX'] * 4 * np.pi * idmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)
idmergedf['PAB_L_ERR'] = idmergedf['PAB_FLUX_ERR'] * 4 * np.pi * idmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)

idmergedfs = pd.merge(GSdf, GStabledf , how = 'inner' , left_on = 'ID' , right_on = 'id3DHST')
idmergedfs.drop('RA_y', axis=1, inplace=True)
idmergedfs.drop('DEC_y', axis=1, inplace=True)
idmergedfs['DL'] = cosmo.luminosity_distance(idmergedfs['Z_50'])
idmergedfs['PAB_L_ERR'] = idmergedfs['PAB_FLUX_ERR'] * 4 * np.pi * idmergedfs['DL']**2 * 9.521 * 10**(48) * 10**(-17)
idmergedfs['PAB_L'] = idmergedfs['PAB_FLUX'] * 4 * np.pi * idmergedfs['DL']**2 * 9.521 * 10**(48) * 10**(-17)
idmergedfs['restalt_restUjohnson'] = idmergedfs['restUjohnson'] 
idmergedfs['restalt_restVjohnson'] = idmergedfs['restVjohnson']

fullmergedf = pd.concat([idmergedf , idmergedfs] , sort = False)
fullmergedf.drop('restUjohnson', axis=1, inplace=True)
fullmergedf.drop('restVjohnson', axis=1, inplace=True)
x = np.genfromtxt('best-fit_sigma1.dat', dtype = None)
sigma1df = pd.DataFrame(x)
fullmergedf['SIGMA_1'] = sigma1df['f1']
sample = np.unique([19221 , 15610 , 34456 , 34157 , 33397 , 33511 , 34368 , 34077 , 33249 , 45518 , 49771 , 37720 , 41882 , 27438 ,
 27549 , 26639 , 26696 , 25632 , 43071 , 39634 , 44465 , 37683 , 19221 , 15610 , 18157 , 34456 , 34157 , 33511 , 34368 , 35455 , 24611 , 33249 ,
19221 , 15610 , 18157 , 34456 , 33511 , 34368 , 35455 , 24611 , 37720 , 41882 , 27438 , 26639 , 26696 , 26646 , 21693 , 42593 , 35433 , 39564 , 27535 , 26559])
print(len(sample))
print(sample)
fullmergebaddf = fullmergedf[fullmergedf.ID.isin(sample) == False]
fullmergedf = fullmergedf[fullmergedf.ID.isin(sample) == True]

fullmergedf['PAB_SFR'] = np.log10(fullmergedf['PAB_L'].astype(float)) - 40.02
fullmergedf['PAB_SFR_ERR']= .434*fullmergedf['PAB_L_ERR']/fullmergedf['PAB_L']
fullmergedf['UV_SFR_ERR']= .434*fullmergedf['ir_SFR-UV_corr_Error']/fullmergedf['ir_SFR-UV_corr']
#fullmergedf['PAB_SFR_EX'] = fullmergedf['PAB_SFR'] - fullmergedf['ir_SFR-UV_corr'] 
fullmergedf['PAB_SFR_EX2'] = fullmergedf['PAB_SFR'] - np.log10(fullmergedf['ir_SFR-UV_corr'])
fullmergedf['PAB_SFR_EX2_ladder'] = fullmergedf['PAB_SFR'] - np.log10(fullmergedf['ir_SFR-ladder_total'])
fullmergedf['PAB_SFR_EX2_ERR'] = np.sqrt(fullmergedf['PAB_SFR_ERR'].astype(None)**2 + fullmergedf['UV_SFR_ERR'].astype(None)**2)
fullmergedf['PAB_SFR_LL'] = np.log10((min(fullmergedf['PAB_FLUX']) * 4 * np.pi * fullmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)).astype(float)) - 40.02
fullmergedf['PAB_SFR_EX2_LL'] = fullmergedf['PAB_SFR_LL'] - np.log10(fullmergedf['ir_SFR-UV_corr'])
fullmergedf['PAB_SFR_LL_1SIGMA'] = np.log10((min(fullmergedf['PAB_FLUX_ERR']) * 4 * np.pi * fullmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)).astype(float)) - 40.02
fullmergedf['PAB_SFR_EX_LL_1SIGMA'] = fullmergedf['PAB_SFR_LL_1SIGMA'] - np.log10(fullmergedf['ir_SFR-UV_corr']) 

32
[15610 18157 19221 21693 24611 25632 26559 26639 26646 26696 27438 27535
 27549 33249 33397 33511 34077 34157 34368 34456 35433 35455 37683 37720
 39564 39634 41882 42593 43071 44465 45518 49771]


In [8]:
fullmergebaddf['PAB_SFR'] = np.log10(fullmergebaddf['PAB_L'].astype(float)) - 40.02
fullmergebaddf['PAB_SFR_ERR']= .434*fullmergebaddf['PAB_L_ERR']/fullmergebaddf['PAB_L']
fullmergebaddf['UV_SFR_ERR']= .434*fullmergebaddf['ir_SFR-UV_corr_Error']/fullmergebaddf['ir_SFR-UV_corr']
fullmergebaddf['PAB_SFR_EX2'] = fullmergebaddf['PAB_SFR'] - np.log10(fullmergebaddf['ir_SFR-UV_corr'])
fullmergebaddf['PAB_SFR_EX2_ladder'] = fullmergebaddf['PAB_SFR'] - np.log10(fullmergebaddf['ir_SFR-ladder_total'])
fullmergebaddf['PAB_SFR_EX2_ERR'] = np.sqrt(fullmergebaddf['PAB_SFR_ERR'].astype(None)**2 + fullmergebaddf['UV_SFR_ERR'].astype(None)**2)

In [9]:
samea280 = [49771 , 39564 , 26559]
fullmergedf = fullmergedf[fullmergedf.ID.isin(samea280) == False]

In [10]:
fullmergedf.to_csv('fullmergePAB.csv',index=False)
fullmergebaddf.to_csv('fullmergebadPAB.csv',index=False)

In [11]:
#idmergedf['SIGMA_1'] = sigma1df['f1']
x = np.genfromtxt('tkrs_gdnmatched.dat', dtype = None)
hadf = pd.DataFrame(x)
hadf = hadf.rename(columns = {'f0':'3DHST_ID','f1':'offset' , 'f2':'Ben_ID' , 'f3':'z' , 'f4':'rmag' , 'f5':'hkmag' , 'f6':'sigkms' , 'f7':'esigkms' , 'f8':'loglum3727' , 'f9':'eloglum3727' , 'f10':'loglum4340' , 'f11':'eloglum4340','f12':'loglum4861' , 'f13':'eloglum4861' , 'f14':'loglum5007' , 'f15':'eloglum5007' , 'f16':'loglum6563' , 'f17':'eloglum6563' , 'f18':'loglum6583' , 'f19':'eloglum6583'})
hamergedf = pd.merge(fullmergedf, hadf , how = 'inner' , left_on = 'ID' , right_on = '3DHST_ID')
#hamergedf.drop(4, axis=0, inplace=True)
hamergedf.drop(12, axis=0, inplace=True)
# hamergedf.drop(10, axis=0, inplace=True)
hamergedf['HA_L'] = 10**hamergedf['loglum6563']
hamergedf['HA_L_ERR'] = (1/.434)*10**hamergedf['loglum6563'] * hamergedf['eloglum6563']
hamergedf['HB_L'] = 10**hamergedf['loglum4861']
hamergedf['HB_L_ERR'] = (1/.434)*10**hamergedf['loglum4861'] * hamergedf['eloglum4861']
hamergedf['HA_FLUX'] = 10**hamergedf['loglum6563']/(4*np.pi*hamergedf['DL']**2*9.521 * 10**(48))
hamergedf['HA_FLUX_ERR'] = hamergedf['HA_FLUX']*np.sqrt((hamergedf['HA_L_ERR']/hamergedf['HA_L'])**2)
hamergedf['HB_FLUX'] = 10**hamergedf['loglum4861']/(4*np.pi*hamergedf['DL']**2*9.521 * 10**(48))
hamergedf['HB_FLUX_ERR'] = hamergedf['HB_FLUX']*np.sqrt((hamergedf['HB_L_ERR']/hamergedf['HB_L'])**2)
hamergedf['PAB_FLUX'] = hamergedf['PAB_FLUX']*10**(-17)
hamergedf['PAB_FLUX_ERR'] = hamergedf['PAB_FLUX_ERR']*10**(-17)
hamergedf['PAB_L'] = hamergedf['PAB_FLUX'] * 4 * np.pi * hamergedf['DL']**2 * 9.521 * 10**(48)
hamergedf['HA_SFR'] = hamergedf['loglum6563'] - 41.27
hamergedf['PAB_SFR'] = np.log10(hamergedf['PAB_L'].astype(float)) - 40.02
hamergedf['PABHA_EXCESS'] = hamergedf['PAB_SFR'] - hamergedf['HA_SFR']
#display(hamergedf)
x = np.genfromtxt('pabhahb_dc_balmer.dat', dtype = None)
balmerdf = pd.DataFrame(x)
balmerdf = balmerdf.rename(columns = {'f0':'FIELD' , 'f1':'ID' , 'f2':'PAB_FLUX_DC_BALMER' , 'f3':'PAB_FLUX_DC_BALMER_ERR' , 'f4':'HA_FLUX_DC_BALMER' , 'f5':'HA_FLUX_DC_BALMER_ERR' , 'f6':'HB_FLUX_DC_BALMER' , 'f7':'HB_FLUX_DC_BALMER_ERR'})
#display(balmerdf)

x = np.genfromtxt('pabhahb_dc_pabha.dat', dtype = None)
pabhadcdf = pd.DataFrame(x)
pabhadcdf = pabhadcdf.rename(columns = {'f0':'FIELD' , 'f1':'ID' , 'f2':'PAB_FLUX_DC_PABHA' , 'f3':'HA_FLUX_DC_PABHA' , 'f4':'HB_FLUX_DC_PABHA'})
# display(pabhadcdf)

dcmergedf = pd.merge(hamergedf, balmerdf , how = 'inner' , left_on = 'ID' , right_on = 'ID')
dcmergedf = pd.merge(dcmergedf, pabhadcdf , how = 'inner' , left_on = 'ID' , right_on = 'ID')
#dcmergedf.loc[4 , 'PAB_FLUX_DC_BALMER'] = 8.37e-16
dcmergedf['HA_L_DC_BALMER'] = dcmergedf['HA_FLUX_DC_BALMER']*(4*np.pi*hamergedf['DL']**2*9.521 * 10**(48))
dcmergedf['HA_L_DC_BALMER_ERR'] = dcmergedf['HA_FLUX_DC_BALMER_ERR']*(4*np.pi*hamergedf['DL']**2*9.521 * 10**(48))
dcmergedf['HA_L_DC_PABHA'] = dcmergedf['HA_FLUX_DC_PABHA']*(4*np.pi*hamergedf['DL']**2*9.521 * 10**(48))
dcmergedf['PAB_L_DC_BALMER'] = dcmergedf['PAB_FLUX_DC_BALMER']*(4*np.pi*hamergedf['DL']**2*9.521 * 10**(48))
dcmergedf['PAB_L_DC_BALMER_ERR'] = dcmergedf['PAB_FLUX_DC_BALMER_ERR']* 4 * np.pi * dcmergedf['DL']**2 * 9.521 * 10**(48)
# dcmergedf['PAB_FLUX_DC_BALMER'].iloc[10] = dcmergedf['PAB_FLUX'].iloc[10]
# dcmergedf['HA_FLUX_DC_BALMER'].iloc[10] = dcmergedf['HA_FLUX'].iloc[10]
# dcmergedf['HB_FLUX_DC_BALMER'].iloc[10] = dcmergedf['HB_FLUX'].iloc[10]
dcmergedf['HA_SFR_DC_BALMER'] = np.log10(dcmergedf['HA_L_DC_BALMER'].astype(float)) - 41.27
dcmergedf['HA_SFR_DC_BALMER_ERR'] = .434*dcmergedf['HA_L_DC_BALMER_ERR']/dcmergedf['HA_L_DC_BALMER']
dcmergedf['PAB_SFR_DC_BALMER'] = np.log10(dcmergedf['PAB_L_DC_BALMER'].astype(float)) - 40.02
dcmergedf['PAB_SFR_DC_BALMER_ERR'] = .434*dcmergedf['PAB_L_DC_BALMER_ERR']/dcmergedf['PAB_L_DC_BALMER']
dcmergedf['PABHA_EXCESS_DC_BALMER'] = dcmergedf['PAB_SFR_DC_BALMER'] - dcmergedf['HA_SFR_DC_BALMER']
dcmergedf['PABHA_EXCESS_DC_BALMER_ERR'] = np.sqrt(dcmergedf['PAB_SFR_DC_BALMER_ERR'].astype(None)**2 + dcmergedf['HA_SFR_DC_BALMER_ERR'].astype(None)**2)
dcmergedf['BALMER_ERR'] = dcmergedf['HA_FLUX']/dcmergedf['HB_FLUX'] *np.sqrt((dcmergedf['HA_FLUX_ERR'].astype(None)/dcmergedf['HA_FLUX'].astype(None))**2 + (dcmergedf['HB_FLUX_ERR'].astype(None)/dcmergedf['HB_FLUX'].astype(None))**2)
dcmergedf['PABHA_ERR'] = dcmergedf['PAB_FLUX']/dcmergedf['HA_FLUX'] *np.sqrt((dcmergedf['PAB_FLUX_ERR'].astype(None)/dcmergedf['PAB_FLUX'].astype(None))**2 + (dcmergedf['HA_FLUX_ERR'].astype(None)/dcmergedf['HA_FLUX'].astype(None))**2)
dcmergedf['BALMER_DC_ERR'] = dcmergedf['HA_FLUX_DC_BALMER']/dcmergedf['HB_FLUX_DC_BALMER'] *np.sqrt((dcmergedf['HA_FLUX_DC_BALMER_ERR'].astype(None)/dcmergedf['HA_FLUX_DC_BALMER'].astype(None))**2 + (dcmergedf['HB_FLUX_DC_BALMER_ERR'].astype(None)/dcmergedf['HB_FLUX_DC_BALMER'].astype(None))**2)
dcmergedf['PABHA_DC_ERR'] =  dcmergedf['PAB_FLUX_DC_BALMER']/dcmergedf['HA_FLUX_DC_BALMER'] *np.sqrt((dcmergedf['PAB_FLUX_DC_BALMER_ERR'].astype(None)/dcmergedf['PAB_FLUX_DC_BALMER'].astype(None))**2 + (dcmergedf['HA_FLUX_DC_BALMER_ERR'].astype(None)/dcmergedf['HA_FLUX_DC_BALMER'].astype(None))**2)

In [12]:
dcmergedf.to_csv('dcmergePAB.csv',index=False)

In [14]:
dust = np.genfromtxt('dustratios.dat', dtype = None)
dustdf = pd.DataFrame(dust, columns = ['AV' , 'HAHB' , 'PABHA'])
dustdf.to_csv('dustdfPAB.csv' , index = False)
attdf = pd.merge(hamergedf, GNtabledf , how = 'inner' , left_on = 'ID' , right_on = 'id3DHST')
attdf.to_csv('attdfPAB.csv' , index = False)
dustcurves = np.genfromtxt('dustcurves.dat', dtype = None)
dustcurvesdf = pd.DataFrame(dustcurves, columns = ['HAHB' , 'PABHA_CALZ00' , 'AV_CALZ00' , 'PABHA_FITZ99' , 'AV_FITZ99' , 'PABHA_GORD03' , 'AV_GORD03'])
dustcurvesdf.to_csv('dustcurvesdf.csv' , index = False)

In [15]:
hdul = fits.open('grizli_v2.1.fits')
zcrit = .3
data_dict= dict()
for obj in hdul[1].data:
    for key in ['FIELD' , 'ID' , 'RA', 'DEC' ,  'Z_50']:
        data_dict.setdefault(key, list()).append(obj[key])
cleardf = pd.DataFrame(data_dict)

cleardf = cleardf[(cleardf.Z_50 >= 0)&(cleardf.Z_50 <= zcrit)]

clearGNdf = cleardf[cleardf['FIELD'] == 'GN1']
clearGNdf = clearGNdf.append(cleardf[cleardf['FIELD'] == 'GN2'])
clearGNdf = clearGNdf.append(cleardf[cleardf['FIELD'] == 'GN3'])
clearGNdf = clearGNdf.append(cleardf[cleardf['FIELD'] == 'GN4'])
clearGNdf = clearGNdf.append(cleardf[cleardf['FIELD'] == 'GN5'])
clearGNdf = clearGNdf.append(cleardf[cleardf['FIELD'] == 'GN7'])

clearGSdf = cleardf[cleardf['FIELD'] == 'GS1']
clearGSdf = clearGSdf.append(cleardf[cleardf['FIELD'] == 'GS2'])
clearGSdf = clearGSdf.append(cleardf[cleardf['FIELD'] == 'GS3'])
clearGSdf = clearGSdf.append(cleardf[cleardf['FIELD'] == 'GS4'])
clearGSdf = clearGSdf.append(cleardf[cleardf['FIELD'] == 'GS5'])
clearGSdf = clearGSdf.append(cleardf[cleardf['FIELD'] == 'ERSPRIME'])

clearnmergedf = pd.merge(clearGNdf, GNtabledf , how = 'inner' , left_on = 'ID' , right_on = 'id3DHST')
clearnmergedf.drop('RA_y', axis=1, inplace=True)
clearnmergedf.drop('DEC_y', axis=1, inplace=True)
clearnmergedf['DL'] = cosmo.luminosity_distance(clearnmergedf['Z_50'])
#clearnmergedf['PAB_L'] = clearnmergedf['PAB_FLUX'] * 4 * np.pi * clearnmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)

clearsmergedf = pd.merge(clearGSdf, GStabledf , how = 'inner' , left_on = 'ID' , right_on = 'id3DHST')
clearsmergedf.drop('RA_y', axis=1, inplace=True)
clearsmergedf.drop('DEC_y', axis=1, inplace=True)
clearsmergedf['DL'] = cosmo.luminosity_distance(clearsmergedf['Z_50'])
fullcleardf = pd.concat([clearnmergedf , clearsmergedf] , sort = False)


In [16]:
fullcleardf.to_csv('fullcleardf.csv' , index = False)

In [17]:
sortedclearmergedf = pd.merge(sortedpabdf , fullcleardf , how = 'inner' , left_on = 'ID' , right_on = 'ID' )

In [18]:
sortedclearmergedf["PAB_FLUX_LIMIT"] = sortedclearmergedf["PAB_FLUX_ERR"]
sortedclearmergedf['PAB_L_LIMIT'] = sortedclearmergedf['PAB_FLUX_LIMIT'] * 4 * np.pi * sortedclearmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)
sortedclearmergedf['PAB_SFR_LIMIT'] = np.log10(sortedclearmergedf['PAB_L_LIMIT'].astype(float)) - 40.02
sortedclearmergedf['PAB_SFR_EX2_LIMIT'] = sortedclearmergedf['PAB_SFR_LIMIT'] - np.log10(sortedclearmergedf['ir_SFR-UV_corr'])
sortedclearmergedf['RAND'] = abs(np.random.normal(0 , 1 , len(sortedclearmergedf)))
sortedclearmergedf["PAB_FLUX_LIMIT_RAND"] = sortedclearmergedf["PAB_FLUX_ERR"]*sortedclearmergedf["RAND"]
sortedclearmergedf['PAB_L_LIMIT_RAND'] = sortedclearmergedf['PAB_FLUX_LIMIT_RAND'] * 4 * np.pi * sortedclearmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)
sortedclearmergedf['PAB_SFR_LIMIT_RAND'] = np.log10(sortedclearmergedf['PAB_L_LIMIT_RAND'].astype(float)) - 40.02
sortedclearmergedf['PAB_SFR_EX2_LIMIT_RAND'] = sortedclearmergedf['PAB_SFR_LIMIT_RAND'] - np.log10(sortedclearmergedf['ir_SFR-UV_corr'])

In [19]:
sortedclearmergedf.to_csv('sortedclearmergedf.csv' , index = False)

In [20]:
limitdf = pd.concat([fullmergedf , sortedclearmergedf])
limitdf.reset_index(drop = True,inplace=True)

In [21]:
limitdf['PAB_SFR_EXCESS_LIMIT_RAND'] = limitdf['PAB_SFR_EX2'][abs(limitdf.PAB_SFR_EX2) >= 0]
limitdf['PAB_SFR_EXCESS_LIMIT_RAND'][abs(limitdf.PAB_SFR_EX2_LIMIT_RAND) >= 0] = limitdf['PAB_SFR_EX2_LIMIT_RAND'][abs(limitdf.PAB_SFR_EX2_LIMIT_RAND) >= 0]
limitdf = limitdf.sort_values(by='td_lmass')
# # limitdf['PAB_SFR_EX2'][abs(limitdf.PAB_SFR_EX2_LIMIT_RAND) >= 0] = limitdf['PAB_SFR_EX2_LIMIT_RAND']
limitdf.reset_index(drop = True,inplace=True)
limitdf = limitdf.drop_duplicates(subset=['ID'], keep='first')
display(limitdf)

,FIELD,ID,RA_x,DEC_x,Z_50,Z_ERR,PAB_FLUX,PAB_FLUX_ERR,OBS_PAB,id3DHST,...,PAB_FLUX_LIMIT,PAB_L_LIMIT,PAB_SFR_LIMIT,PAB_SFR_EX2_LIMIT,RAND,PAB_FLUX_LIMIT_RAND,PAB_L_LIMIT_RAND,PAB_SFR_LIMIT_RAND,PAB_SFR_EX2_LIMIT_RAND,PAB_SFR_EXCESS_LIMIT_RAND
0,NaN,41139,53.129122,-27.718296,NaN,0.093696,9.183679,0.621907,12973.684349,41139,...,0.621907,2037643554788830195429007916420562944.0,-3.710872,-4.682494,0.235065,0.146188,478978234209605792505437051408089088.0,-4.339684,-5.311306,-5.311306
1,NaN,41160,53.130804,-27.717711,NaN,0.041602,0.101169,0.439210,13748.816469,41160,...,0.439210,57474458231615098891698885855417466880.0,-2.260525,0.560921,0.556612,0.244469,31990949395880222734134227831248388096.0,-2.514973,0.306473,0.306473
2,NaN,37545,53.197312,-27.736315,NaN,0.347463,0.334175,0.387822,13962.635624,37545,...,0.387822,78617583416955668762594243207476805632.0,-2.124480,1.376757,1.207545,0.468313,94934235461406919432346773406649155584.0,-2.042577,1.458660,1.458660
3,NaN,26881,53.180778,-27.786255,NaN,0.024785,0.229275,0.308107,14819.358877,26881,...,0.308107,208860266422471459579322992871201570816.0,-1.700144,0.356552,0.746128,0.229887,155836552306218907809840078139124875264.0,-1.827331,0.229366,0.229366
4,NaN,33042,53.133103,-27.758703,NaN,0.208233,0.404118,0.565307,13017.631077,33042,...,0.565307,3079142148105443303018557634441641984.0,-3.531570,-4.184665,0.831084,0.469817,2559025531619238701968382058158882816.0,-3.611925,-4.265020,-4.265020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,NaN,33015,189.282226,62.314258,NaN,0.001463,53.314569,1.990500,15416.308291,33015,...,1.990500,2408466279676888139327059844195093053440.0,-0.638259,0.140559,1.161932,2.312825,2798473719387584710191854689699781148672.0,-0.573079,0.205739,0.205739
288,NaN,19146,53.143835,-27.813467,NaN,0.004738,218.857585,16.852526,15536.853897,19146,...,16.852526,22576958535776746560509547046509966524416.0,0.333665,-0.635170,0.307294,5.178676,6937758777368445374700370166980961894400.0,-0.178781,-1.147616,-1.147616
289,NaN,44959,53.139785,-27.700114,NaN,0.005971,6.039735,0.676982,12958.744936,44959,...,0.676982,1804569657641045188516705427026608128.0,-3.763626,-4.012744,0.530677,0.359259,957643292316913837895025978368851968.0,-4.038796,-4.287914,-4.287914
293,NaN,42866,53.150016,-27.708661,NaN,0.003998,42.115839,1.505243,12931.370021,42866,...,1.505243,2576736955765152389590631390786879488.0,-3.608930,-3.604193,0.142657,0.214734,367590621717596414533423997988634624.0,-4.454636,-4.449899,-4.449899


In [ ]:
limitdf.to_csv('limitdf.csv' , index = False)

In [22]:
limitpabdcdf = pd.read_csv('limitpabdcdf_avcor_csv.csv')
samplepabdcdf = pd.read_csv('samplepabdcdf_avcor.csv')

In [23]:
dcsampledf = pd.merge(fullmergedf , samplepabdcdf , how = 'inner' , left_on = 'ID' , right_on = 'id' )

In [45]:
dcsampledf.to_csv('dcsampledf.csv' , index = False)

In [24]:
limitpabdcdf = limitpabdcdf.drop_duplicates(subset=['id'], keep='first')
dclimitdf = pd.merge(limitdf , limitpabdcdf , how = 'inner' , left_on = 'ID' , right_on = 'id' )

In [28]:
dclimitdf['SFR_RAND'] = dclimitdf['sfrratio_calz']*dclimitdf['RAND']
dclimitdf = dclimitdf[abs(dclimitdf.SFR_RAND) > 0]

In [29]:
lmlimitalldc = linmix.LinMix(dclimitdf['td_lmass'], dclimitdf['SFR_RAND'] , [.3]*len(dclimitdf['td_lmass']) , [.3]*len(dclimitdf['td_lmass']) , K = 2, seed = 2)
lmlimitalldc.run_mcmc(silent = True)

In [30]:
# dclimitdf['ROLL_STD'] = dclimitdf['PAB_SFR_EXCESS_LIMIT_RAND'].rolling(10).std()
dclimitdf['ROLL_MAD'] = abs(dclimitdf['PAB_SFR_EXCESS_LIMIT_RAND']).rolling(50 , min_periods = 1).median()
dclimitdf['EXCESS_DIS_FIT'] = dclimitdf['PAB_SFR_EXCESS_LIMIT_RAND'] - (dclimitdf['td_lmass']*np.mean(lmlimitalldc.chain['beta']) + np.mean(lmlimitalldc.chain['alpha']))
dclimitdf['ROLL_MAD_FIT_TOP'] = abs(dclimitdf['EXCESS_DIS_FIT']).rolling(50 , min_periods = 1).median() + (dclimitdf['td_lmass']*np.mean(lmlimitalldc.chain['beta']) + np.mean(lmlimitalldc.chain['alpha']))
dclimitdf['ROLL_MAD_FIT_BOT'] = -abs(dclimitdf['EXCESS_DIS_FIT']).rolling(50 , min_periods = 1).median() + (dclimitdf['td_lmass']*np.mean(lmlimitalldc.chain['beta']) + np.mean(lmlimitalldc.chain['alpha']))
#dclimitdf['ROLL_MAD_FIT'] = (1/50)*dclimitdf['ROLL_MAD_FIT']

In [31]:
dclimitdf.to_csv('dclimitdf.csv' , index = False)

In [35]:
dclimitbetanegdf = dclimitdf[dclimitdf.ir_A_UV_280 < 1.2].copy()
dclimitbetanegdf = dclimitbetanegdf[dclimitbetanegdf.sfrratio != -9.99]
dclimitbetanegdf['EX_DC_RAND'] = dclimitbetanegdf['sfrratio_calz']*dclimitbetanegdf['RAND']
dclimitbetanegdf = dclimitbetanegdf[abs(dclimitbetanegdf.EX_DC_RAND) > 0]

In [36]:
dclmlimitbetaneg = linmix.LinMix(dclimitbetanegdf['td_lmass'] , dclimitbetanegdf['EX_DC_RAND'] , [.3]*len(dclimitbetanegdf['td_lmass']) , [.3]*len(dclimitbetanegdf['td_lmass']) , K = 2, seed = 2)
dclmlimitbetaneg.run_mcmc(silent = True)

In [37]:
dclimitbetanegdf['ROLL_MAD'] = abs(dclimitbetanegdf['EX_DC_RAND']).rolling(50 , min_periods = 1).median()
dclimitbetanegdf['EXCESS_DIS_FIT_BETA_NEG'] = dclimitbetanegdf['EX_DC_RAND'] - (dclimitbetanegdf['td_lmass'] * np.mean(dclmlimitbetaneg.chain['beta']) + np.mean(dclmlimitbetaneg.chain['alpha']))
dclimitbetanegdf['ROLL_MAD_FIT_TOP'] = abs(dclimitbetanegdf['EXCESS_DIS_FIT_BETA_NEG']).rolling(50 , min_periods = 1).median() + (dclimitbetanegdf['td_lmass'] * np.mean(dclmlimitbetaneg.chain['beta']) + np.mean(dclmlimitbetaneg.chain['alpha']))
dclimitbetanegdf['ROLL_MAD_FIT_BOT'] = -abs(dclimitbetanegdf['EXCESS_DIS_FIT_BETA_NEG']).rolling(50 , min_periods = 1).median() + (dclimitbetanegdf['td_lmass'] * np.mean(dclmlimitbetaneg.chain['beta']) + np.mean(dclmlimitbetaneg.chain['alpha']))
#display(limitbetanegdf)

In [39]:
dclimitbetanegdf.to_csv('dclimitbetanegdf.csv' , index = False)

In [46]:
morphdf = fullmergedf[fullmergedf['gf_f_j'] == 0].copy()
morphdf.to_csv('morphdf.csv' , index = False)
dcmorphdf = pd.merge(morphdf , dcsampledf , how = 'inner' , left_on = 'ID' , right_on = 'id')
dcmorphdf.to_csv('dcmorphdf.csv' , index = False)